# Score Scenarios

Run the cells below to score the configured scenarios. Any unscored scenarios will be scored.

To re-score a scenario, open its corresponding file geodatabase and delete its `scores` and `scores_summary` table.

In [ ]:
import sys
import os
import arcpy

import pandas as pd
from arcgis.features import SpatialDataFrame

src = os.path.join(os.path.abspath("."), 'src')
if src not in sys.path:
    sys.path.append(src)
    
from ato_tools import network
from ato_tools import scoring

if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(scoring)
    importlib.reload(network)

base_path = os.path.abspath(".")

In [ ]:
modes = list()
for file in os.listdir(r'scenario'):
    d = os.path.join(base_path, r'scenario', file)
    if os.path.isdir(d) and not d.endswith('.gdb'):
        modes.append(file)

to_score = list()
for mode in ['Cycling', 'Driving', 'Transit']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(base_path, scenario_folder, file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == False:
                print(d)
                to_score.append((d, mode))

In [ ]:
# score unscored networks
arcpy.env.addOutputsToMap = False

for scenario in to_score:
    gdb, mode = scenario
    scoring.score(gdb, mode = mode, test = False)

arcpy.env.addOutputsToMap = True

If the loop above fails due to an invalid network, you can rebuild the network by running `network.build()` on the full network dataset path, e.g. `network.build(r'scenario\Transit\ogden_local_bus.gdb\NetworkDataset\NetworkDataset_ND')`

## Tabulate Scores Across Scenarios

The cells below read in the scores from all scored scenarios and combined into a single summary table (`scenario\scenario_scores.csv`)

Scores for individual projects can be extracted from the `scores_summary` table within each file geodatabase.

In [ ]:
field_list = ['CO_TAZID', 'HH', 'JOB', 'accessible_jobs', 'accessible_hh', 'ato']

arr = arcpy.da.TableToNumPyArray(r'baseline.gdb\auto_baseline_summary', field_list)
baseline_auto = pd.DataFrame(arr, columns = field_list)

arr = arcpy.da.TableToNumPyArray(r'baseline.gdb\transit_baseline_summary', field_list)
baseline_transit = pd.DataFrame(arr, columns = field_list)

In [ ]:
summary_long = pd.DataFrame(columns = ['Name', 'Mode', 'hh', 'jobs', 'ato'])

vals = {
    "Name":"Baseline",
    "Mode": "Auto",
    "hh": baseline_auto['accessible_hh'].sum(),
    "jobs": baseline_auto['accessible_jobs'].sum(),
    "ato": baseline_auto['ato'].sum()
}
summary_long = summary_long.append(vals, ignore_index=True)

vals = {
    "Name": "Baseline",
    "Mode": "Transit",
    "hh": baseline_transit['accessible_hh'].sum(),
    "jobs": baseline_transit['accessible_jobs'].sum(),
    "ato": baseline_transit['ato'].sum()
}
summary_long = summary_long.append(vals, ignore_index=True)

In [ ]:
scenarios = list()

for mode in ['Driving', 'Transit', 'Cycling']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(scenario_folder, file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == True:

                scenarios.append({
                                "name": file[:-4],
                                "mode": mode,
                                "gdb_path": d
                            })

In [ ]:
for scenario in scenarios:
    arr = arcpy.da.TableToNumPyArray(scenario['gdb_path'] + r'\scores_summary', field_list)
    scores = pd.DataFrame(arr, columns = field_list)
    
    mode = scenario['mode']
    baseline = baseline_auto if mode == 'Driving' else baseline_transit
    
    df = pd.merge(
        baseline, 
        scores, 
        on='CO_TAZID', 
        how="inner",
        suffixes=("_before", "_after")
    )
    df['diff_hh'] = df['accessible_hh_after'] - df['accessible_hh_before']
    df['diff_jobs'] = df['accessible_jobs_after'] - df['accessible_jobs_before']
    df['diff_ato'] = df['ato_after'] - df['ato_before']
    vals = {
        "Name": scenario['name'],
        'Mode': mode,
        "hh": df['diff_hh'].sum(),
        "jobs": df['diff_jobs'].sum(),
        "ato": df['diff_hh'].sum()
    }
    
    summary_long = summary_long.append(vals, ignore_index=True)

In [ ]:
summary_long

In [ ]:
summary_long.to_csv(r'scenario\scenario_scores.csv')